<a href="https://colab.research.google.com/github/Akhil122005/techsophy/blob/main/deployment_risk_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn scikit-learn pandas joblib nest_asyncio

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from joblib import dump

# Sample dataset
data = {
    'lines_changed': [10, 500, 30, 800, 15],
    'services_impacted': [1, 4, 2, 5, 1],
    'weekend_deploy': [0, 1, 0, 1, 0],
    'rollback': [0, 1, 0, 1, 0]
}

df = pd.DataFrame(data)
X = df.drop('rollback', axis=1)
y = df['rollback']

# Train model
model = RandomForestClassifier()
model.fit(X, y)

# Save model
dump(model, 'risk_model.joblib')


['risk_model.joblib']

In [ ]:
from fastapi import FastAPI
from fastapi.responses import HTMLResponse
from joblib import load
import numpy as np
import pandas as pd
import random

app = FastAPI(title="Deployment Risk Assessment API")
model = load('risk_model.joblib')

@app.get("/", response_class=HTMLResponse)
def home():
    return """
    <html>
        <head><title>Deployment Risk Predictor</title></head>
        <body style="font-family:Arial;">
            <h1>🚀 Deployment Risk Assessment App</h1>
            <p>This system predicts the risk of a deployment and triggers a rollback if needed.</p>
            <p>👉 Visit <a href='/docs'>/docs</a> to test the API using Swagger UI.</p>
            <p>👨‍💻 Author: <b>Kura Akhilesh</b></p>
        </body>
    </html>
    """

def predict_risk(lines_changed, services_impacted, weekend_deploy):
    features = pd.DataFrame([{
        'lines_changed': lines_changed,
        'services_impacted': services_impacted,
        'weekend_deploy': weekend_deploy
    }])
    return model.predict_proba(features)[0][1]

def get_deployment_metrics():
    return {
        'latency': random.uniform(50, 500),
        'error_rate': random.uniform(0.01, 0.2),
    }

def is_anomalous(metrics):
    return metrics['latency'] > 300 or metrics['error_rate'] > 0.1

def trigger_rollback():
    print("⚠️ Rollback triggered! Reverting deployment...")

@app.post("/deploy")
def deploy_app(lines_changed: int, services_impacted: int, weekend_deploy: int):
    risk = predict_risk(lines_changed, services_impacted, weekend_deploy)
    print(f"Predicted Risk: {risk:.2f}")

    if risk > 0.8:
        trigger_rollback()
        return {"status": "rollback", "reason": "High predicted risk"}

    metrics = get_deployment_metrics()
    if is_anomalous(metrics):
        trigger_rollback()
        return {"status": "rollback", "reason": "Anomaly in metrics"}

    return {"status": "deployed", "risk": round(risk, 2), "metrics": metrics}


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

nest_asyncio.apply()

# Open ngrok tunnel
public_url = ngrok.connect(8000)
print("🚀 Public FastAPI URL:", public_url)

# Start FastAPI server
uvicorn.run(app, host="0.0.0.0", port=8000)


🚀 Public FastAPI URL: NgrokTunnel: "https://9efc-34-118-242-131.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [1146]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     103.255.144.74:0 - "GET / HTTP/1.1" 200 OK
INFO:     103.255.144.74:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     103.255.144.74:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     103.255.144.74:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     103.255.144.74:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     103.255.144.74:0 - "GET /openapi.json HTTP/1.1" 200 OK
Predicted Risk: 0.53
INFO:     103.255.144.74:0 - "POST /deploy?lines_changed=400&services_impacted=3&weekend_deploy=1 HTTP/1.1" 200 OK
Predicted Risk: 0.53
⚠️ Rollback triggered! Reverting deployment...
INFO:     103.255.144.74:0 - "POST /deploy?lines_changed=400&services_impacted=3&weekend_deploy=1 HTTP/1.1" 200 OK
INFO:     103.255.144.74:0 - "GET /deploy?lines_changed=400&services_impacted=3&weekend_deploy=1 HTTP/1.1" 405 Method Not Allowed
INFO:     103.255.144.74:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     103.255.144.74:0 - "GET /openapi.json HTTP/1.1" 200 OK
Predicted Risk: 0.53
INFO:     103.255.144.74:0 - "POST /depl

In [ ]:
deploy_app(lines_changed=400, services_impacted=3, weekend_deploy=1)


Predicted Risk: 0.65
⚠️ Rollback triggered! Reverting deployment...


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


{'status': 'rollback', 'reason': 'Anomaly in metrics'}

In [ ]:
!pip install pyngrok


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

In [ ]:
!ngrok config add-authtoken 2zX8Ytzj1e9i5MKvPocYXdz00xH_4qXqPd5Qu8SPSb5c9oWa9


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
